In [1]:
#!pip install gspread
#!pip install oauth2client
#!pip install PyGithub
#!pip install google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib
#!pip install openpyxl  # для работы с .xlsx

## Рандомизация данных из коммерческого источника Google Sheets

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import random
import string
import os
import json  # Добавьте эту строку в начале файла


In [3]:
# Авторизация
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
with open(r'D:\PycharmProjects\config.ini\credentials.json', 'r', encoding='utf-8') as f:
    creds_data = json.load(f)

creds = ServiceAccountCredentials.from_json_keyfile_dict(creds_data, scope)
client = gspread.authorize(creds)

# Открытие таблицы
sheet = client.open_by_key('1draL0IcMP94h3YChWrHMxfI0AG3gTBgNoXO3lqj0jKE').worksheet('product info')

In [4]:
# Чтение данных
data = sheet.get_all_records()
df = pd.DataFrame(data)


In [5]:
# Функции для рандомизации
def random_string(length=8):
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))

def random_number(length=8):
    return ''.join(random.choices(string.digits, k=length))

# Рандомизация нужных колонок
df['Offer ID'] = df['Offer ID'].apply(lambda x: random_string())
df['SKU'] = df['SKU'].apply(lambda x: random_string())
df['Product ID'] = df['Product ID'].apply(lambda x: random_number())
df['Штрихкоды'] = df['Штрихкоды'].apply(lambda x: random_number(13))
df['Название'] = df['Название'].apply(lambda x: f"Product_{random_string(6)}")

# Обновление данных в Google Sheets
sheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1draL0IcMP94h3YChWrHMxfI0AG3gTBgNoXO3lqj0jKE',
 'updatedRange': "'product info'!A1:K320",
 'updatedRows': 320,
 'updatedColumns': 11,
 'updatedCells': 3520}

In [12]:
from github import Github
from github import GithubException


In [13]:
# Чтение токена из файла
with open(r"D:\PycharmProjects\config.ini\.env.github", 'r', encoding='utf-8') as f:
    github_token = f.read().strip()


In [14]:
# Подключение к GitHub
g = Github(github_token)
user = g.get_user()  # Получаем текущего пользователя


In [15]:
# Название нового репозитория
repo_name = "Tools-for-Ozon-pricing-strategy-optimizing"

try:
    # Пытаемся получить репозиторий (если он уже существует)
    repo = user.get_repo(repo_name)
    print(f"Репозиторий '{repo_name}' уже существует.")
except GithubException as e:
    if e.status == 404:
        # Если репозитория нет — создаём его
        print(f"Репозиторий '{repo_name}' не найден. Создаём новый...")
        repo = user.create_repo(
            repo_name,
            description="Оптимизация ценовой стратегии для Ozon",
            private=False,  # False = публичный репозиторий
            auto_init=False  # Не создавать README при создании
        )
        print(f"Репозиторий успешно создан: {repo.html_url}")
    else:
        raise e  # Если ошибка не 404 — пробрасываем дальше


Репозиторий 'Tools-for-Ozon-pricing-strategy-optimizing' не найден. Создаём новый...
Репозиторий успешно создан: https://github.com/vitaly22041977/Tools-for-Ozon-pricing-strategy-optimizing


In [16]:
# Загрузка файла в репозиторий
file_path = "competitive_analysis/randomized_products.csv"
commit_message = "Add randomized product data"

try:
    # Проверяем, существует ли уже файл
    repo.get_contents(file_path)
    print(f"Файл '{file_path}' уже существует. Используйте update_file для изменения.")
except GithubException as e:
    if e.status == 404:
        # Если файла нет — создаём его
        with open('randomized_products.csv', 'r', encoding='utf-8') as file:
            content = file.read()
        
        repo.create_file(
            path=file_path,
            message=commit_message,
            content=content,
            branch="main"  # Указываем ветку (по умолчанию — main/master)
        )
        print(f"Файл успешно загружен в репозиторий!")
    else:
        raise e

Файл успешно загружен в репозиторий!
